In [1]:
import re
import sys
from pathlib import Path
from collections import Counter

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
ROOT = Path('/home/jun0/Projects/emart24')
sys.path.append(str(ROOT))
print(sys.path)

['/home/jun0/Projects/emart24/emart24_codes/RJY', '/home/jun0/.vscode-server/extensions/ms-toolsai.jupyter-2021.8.1236758218/pythonFiles', '/home/jun0/.vscode-server/extensions/ms-toolsai.jupyter-2021.8.1236758218/pythonFiles/lib/python', '/home/jun0/.pyenv/versions/miniconda3-latest/envs/cuda111/lib/python38.zip', '/home/jun0/.pyenv/versions/miniconda3-latest/envs/cuda111/lib/python3.8', '/home/jun0/.pyenv/versions/miniconda3-latest/envs/cuda111/lib/python3.8/lib-dynload', '', '/home/jun0/.local/lib/python3.8/site-packages', '/home/jun0/.pyenv/versions/miniconda3-latest/envs/cuda111/lib/python3.8/site-packages', '/home/jun0/.local/lib/python3.8/site-packages/IPython/extensions', '/home/jun0/.ipython', '/home/jun0/Projects/emart24']


# 분류 체계 확장

In [3]:
# 카테고리 데이터
category = pd.read_excel(ROOT / 'data/category.xlsx')
category.columns = ['KAN_CODE', '전체분류', '현재분류명', '상품예시']

# 상품 마스터 데이터
item = pd.read_csv(ROOT / 'data/prep/02.설명이추가된 상품마스터_20210325.csv', sep='|', dtype=str)
item['점포발주입수수량'] = item['점포발주입수수량'].fillna(0)
item['공급가'] = item['공급가'].fillna(0).apply(lambda x: round(float(x)))
item['매가'] = item['매가'].fillna(0)
item['점포유통기한일수'] = item['점포유통기한일수'].fillna(0)
item = item.astype({'점포발주입수수량': int, '공급가': int, '매가': int, '점포유통기한일수': int})

# shapes
category.shape, item.shape

((3062, 4), (49299, 46))

In [41]:
category.head(2)

,KAN_CODE,전체분류,현재분류명,상품예시
0,1010101,가공식품>조미료>종합조미료>천연/발효조미료,천연/발효조미료,"다시다, 미원, 멸치가루, 버섯가루"
1,1010102,가공식품>조미료>종합조미료>식초,식초,"과일초, 발효초, 감식초, 사과식초, 현미식초, 홍초"


In [30]:
contained = item.fillna('').apply(lambda row: (row['GDS_SLPNT_DSCRT'] in row['GDS_DSCRT']) & bool(row['GDS_DSCRT']) & bool(row['GDS_SLPNT_DSCRT']), axis=1)
contained_reverse = item.fillna('').apply(lambda row: (row['GDS_DSCRT'] in row['GDS_SLPNT_DSCRT']) & bool(row['GDS_DSCRT']) & bool(row['GDS_SLPNT_DSCRT']), axis=1)
only_dscrt = item['GDS_DSCRT'].isna() & ~item['GDS_SLPNT_DSCRT'].isna()
only_slpnt = ~item['GDS_DSCRT'].isna() & item['GDS_SLPNT_DSCRT'].isna()

In [26]:
item[contained][['상품명', 'GDS_DSCRT', 'GDS_SLPNT_DSCRT']]

,상품명,GDS_DSCRT,GDS_SLPNT_DSCRT
315,에이원)스위트롤리팝2000(21W),긴 막대기에 동그란 모양의 회오리 캔디,긴 막대기에 동그란 모양의 회오리 캔디
957,서울장수)장홍삼막걸리750ml,- 면역력에 대한 관심이 많은 시기에 맞는 홍삼을 가미한 막걸리 - 농협한삼인 ...,- 농협한삼인 6년근 홍삼분말을 넣어 홍삼의 풍미를 그대로 느낄 수 있는 막걸리
1601,해피바스)라벤더비누100GX4입(F),- 이마트24 후레쉬 상품 도입,- 이마트24 후레쉬 상품 도입
2777,가야)농장종합세트180ml,"- 구성 : 복숭아180ml×2개, 당근180ml×2개, 알로에180ml×2개, 토...","- 구성 : 복숭아180ml×2개, 당근180ml×2개, 알로에180ml×2개, 토..."
2798,광동)힘찬하루헛개차340ml,"1. 헛개소재 차음료의 no.1 (M/S 60%, 19년 닐슨 헛개음료시장) 2. ...","1. 헛개소재 차음료의 no.1 (M/S 60%, 19년 닐슨 헛개음료시장) 2. ..."
...,...,...,...
49197,엘림)올뉴기찬도시락2편(특판용)테스트,도시락용 테스트 코드로 (발주넣어도 미입고 되오니 참고부탁드립니다) ※발주 불가 상품,도시락용 테스트 코드로 (발주넣어도 미입고 되오니 참고부탁드립니다) ※발주 불가 상품
49268,e)두꺼비젤리초상콤청포도맛(J),진로납품용직납코드,진로납품용직납코드
49269,e)두꺼비젤리초달콤자두맛(J),진로 직납용 코드,진로 직납용 코드
49296,K)88리턴즈보루,K)88리턴즈 제품의 보루 스캔용 코드,K)88리턴즈 제품의 보루 스캔용 코드


In [32]:
item[only_dscrt][['상품명', 'GDS_DSCRT', 'GDS_SLPNT_DSCRT']]

,상품명,GDS_DSCRT,GDS_SLPNT_DSCRT
7975,삼양)삼양라면멀티*5,NaN,[3월 전단(장보기+비타민)대상 상품] ① 삼양식품의 대표 스테디셀러 상품 ② 3월...
9834,삼양)불닭볶음면봉지*5입,NaN,[3월 전단(장보기+비타민)대상 상품] ① 젊은층에 인기 최고인 빅히트 상품 ② 3...
11019,오뚜기)진라면매운맛소컵*6입,NaN,"[3월 전단(장보기+비타민)대상 상품] ① 간편한 6입 입수로, 좋은 가격에 제공하..."
13086,오뚜기)진라면매운맛봉지*5입,NaN,[3월 전단(장보기+비타민)대상 상품] ① 오뚜기 라면의 베스트셀러 상품 ② 3월 ...
28655,델몬트)바나나2입,NaN,과일브랜드로 세계적으로 유명한 델몬트사의 바나나 2입상품입니다.
30896,풀무원)돈코츠라멘217g,NaN,동절기 시즌상품! 진한 돈골 육수에 된장과 쯔유를 넣은 만든 담백하고 감칠맛이 나는...
32929,특판)호두파이,NaN,울산SK특판
47579,K)에쎄체인지시크릿보루,NaN,K)에쎄체인지시크릿


In [33]:
item[only_slpnt][['상품명', 'GDS_DSCRT', 'GDS_SLPNT_DSCRT']]

,상품명,GDS_DSCRT,GDS_SLPNT_DSCRT
25,피코크)부산어묵탕얼큰한맛350g,부산어묵의 쫄깃함을 얼큰한어묵탕에 담은 피코크 상품,NaN
26,피코크)맛있는순대1kg,국내산 돈창에 당면과 신선한 채소로 속을 채운 국민간식,NaN
27,보승)민속족발200g,"국내산 족발을 당귀,감초등을 넣고 2시간 이상 삶아 쫄깃하고 담백한 족발",NaN
32,원두)엘까페딸원두500g,"세 가지 공정무역 커피가 최상의 비율, 최적의 로스팅 포인트로 만들어진 고품질 블렌...",NaN
35,피코크)돈코츠라멘348g,돈골을 진하게 우려내어 뽀얀 국물의 맛이 진하면서 담백한 일본식 전통 돈코츠라멘,NaN
...,...,...,...
49281,종량제봉투 100L,.,NaN
49282,음식물 폐기물 칩 60L,.,NaN
49283,종량제봉투 75L,.,NaN
49290,"대형폐기물 스티커 5,000원권",.,NaN


In [29]:
item[contained_reverse][['상품명', 'GDS_DSCRT', 'GDS_SLPNT_DSCRT']]

,상품명,GDS_DSCRT,GDS_SLPNT_DSCRT
55,에너자이저)맥스AAA2입,에너자이저 국내 판매 1위 건전지,[사이드진열 Best 추천 상품] 에너자이저 국내 판매 1위 건전지
123,에너자이저)맥스AA2입,에너자이저 국내 판매 1위 건전지,[사이드진열 Best 추천 상품] 에너자이저 국내 판매 1위 건전지
315,에이원)스위트롤리팝2000(21W),긴 막대기에 동그란 모양의 회오리 캔디,긴 막대기에 동그란 모양의 회오리 캔디
704,천지개벽100ml,- 트렌드 : 숙취해소음료 카테고리 내 구색 다양화를 통한 고객 니즈 확보가 필요함...,- 트렌드 : 숙취해소음료 카테고리 내 구색 다양화를 통한 고객 니즈 확보가 필요함...
753,하겐다즈)화이트블레스케익,"- 향긋한 그린티 아이스크림과 부드러운 바닐라 아이스크림, 달콤한 위핑크림과 초코렛...","- 향긋한 그린티 아이스크림과 부드러운 바닐라 아이스크림, 달콤한 위핑크림과 초코렛..."
...,...,...,...
49197,엘림)올뉴기찬도시락2편(특판용)테스트,도시락용 테스트 코드로 (발주넣어도 미입고 되오니 참고부탁드립니다) ※발주 불가 상품,도시락용 테스트 코드로 (발주넣어도 미입고 되오니 참고부탁드립니다) ※발주 불가 상품
49268,e)두꺼비젤리초상콤청포도맛(J),진로납품용직납코드,진로납품용직납코드
49269,e)두꺼비젤리초달콤자두맛(J),진로 직납용 코드,진로 직납용 코드
49296,K)88리턴즈보루,K)88리턴즈 제품의 보루 스캔용 코드,K)88리턴즈 제품의 보루 스캔용 코드


In [27]:
item[~contained][['상품명', 'GDS_DSCRT', 'GDS_SLPNT_DSCRT']]

,상품명,GDS_DSCRT,GDS_SLPNT_DSCRT
0,KGB 크랜베리맛,NaN,NaN
1,레돈도)카푸치노125g,1. 20년 빼빼로데이 기획상품 2. 틴 케이스에 웨이퍼롤이 담겨 있는 상품,1. 틴 케이스에 웨이퍼롤이 담겨 있는 상품
2,수입)레돈도(바닐라)150g,NaN,NaN
3,레돈도딸기&크림,NaN,NaN
4,쉬크)퀴트로4중 면도날(4입),NaN,NaN
...,...,...,...
49292,동원)덴마크청미채그린200ml,1. 건강음료에 대한 소비자 관심증가에 따른 과채주스 추가 도입 2. 20가지의 ...,1. 20가지의 과일&채소가 함유된 마시는 샐러드 주스 2. 무농약 수경재배 ‘청미...
49293,롯데)레쓰비그란데아메리카노500ml,1. 지속 성장하는 대용량 커피 시장에 맞춰 중대형 사이즈의 커피음료 출시 2. 아...,1. 인기 브랜드 레쓰비의 중대형 사이즈의 아메리카노 커피 2. 아메리카노 특유의 ...
49294,광동)흑미차500ml,1. 페트형 보리차음료의 지속적인 성장으로 신규차음료 강화 2. 진도산 흑미를 사용...,1. 심리적 안정을 취할 수 있는 건강지향 차음료 매출 상승 2. 진도산 흑미를 사...
49295,시원)틱톡푸딩젤리320g(A),1. 유튜브 등 SNS로의 ASMR 먹방으로 유명한 틱톡젤리 2. 경쟁사 운영상품 ...,"1. 유튜브 등에서 먹방으로 유명한 틱톡젤리 2. 학원가, 주택가 취급권장 3. 젤..."


In [4]:
temp = pd.read_csv(ROOT / 'data/prep/02.설명이추가된 상품마스터_20210325.csv', sep='|', dtype=str)
temp[temp['점포발주입수수량'].isna() | temp['공급가'].isna() | temp['매가'].isna() | temp['점포유통기한일수'].isna()]

,PLU_CD,상품명,대분류명,중분류명,소분류명,카테고리코드,상품속성코드,상품속성명,판매가능여부,시작일자,...,LT,MON_YN,TUE_YN,WED_YN,THU_YN,FRI_YN,SAT_YN,SUN_YN,GDS_DSCRT,GDS_SLPNT_DSCRT
27089,8809129092277,노브랜드)검은콩두유190ml,유제품,우유,두유,2062012004,401,일반,예,20170203,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,합리적인 가격의 검은콩두유,NaN


In [5]:
# 전체 카테고리
all_category = []
temp = pd.DataFrame(category['전체분류'].apply(lambda row: row.split('>')).tolist())
for col in temp.columns:
    all_category += temp[col].unique().tolist()
unique_category = set(all_category)
len(all_category), len(unique_category)

(3576, 3427)

In [6]:
# category의 아이템
category_item = [x for lst in category['상품예시'].apply(lambda row: [x.strip() for x in row.split(',')]).tolist() for x in lst]
# '' 제거
category_item = list(filter(lambda x: x != '', category_item))
category_item_unique = set(category_item)
category_item_unique = list(category_item_unique)[1:]
len(category_item), len(category_item_unique)

(9353, 9140)

In [7]:
# unique 한 상품명
item_unique_bool = ~item['상품명'].duplicated()
# 분류가 겹치는 아이템
overlap = item_unique_bool & (item['대분류명'].isin(unique_category) | item['중분류명'].isin(unique_category) | item['소분류명'].isin(unique_category))
# 단순 일치하는 상품명
exact_match_bool = item_unique_bool & item['상품명'].isin(category_item_unique)
# 상품명에 포함된 상품예시
contain_bool = item_unique_bool & item['상품명'].apply(lambda x: bool([i for i in category_item_unique if i in x]))

item[overlap].shape, item[exact_match_bool].shape, item[contain_bool].shape, item[overlap | contain_bool].shape

((17830, 46), (283, 46), (23738, 46), (31591, 46))

In [8]:
# 상품 설명 보기
item[['GDS_DSCRT', 'GDS_SLPNT_DSCRT']].drop_duplicates().values.tolist()

[[nan, nan],
 ['1. 20년 빼빼로데이 기획상품 2. 틴 케이스에 웨이퍼롤이 담겨 있는 상품', '1. 틴 케이스에 웨이퍼롤이 담겨 있는 상품'],
 ['부산어묵의 쫄깃함을 얼큰한어묵탕에 담은 피코크 상품', nan],
 ['국내산 돈창에 당면과 신선한 채소로 속을 채운 국민간식', nan],
 ['국내산 족발을 당귀,감초등을 넣고 2시간 이상 삶아 쫄깃하고 담백한 족발', nan],
 ['세 가지 공정무역 커피가 최상의 비율, 최적의 로스팅 포인트로 만들어진 고품질 블렌딩 커피', nan],
 ['돈골을 진하게 우려내어 뽀얀 국물의 맛이 진하면서 담백한 일본식 전통 돈코츠라멘', nan],
 ['부산어묵의 쫄깃함을 시원한어묵탕에 담은 피코크 상품', nan],
 ['신선한 버터의 고소함과 부드러움이 그대로 살아있는 버터', nan],
 ['수입과자 카테고리 킬러 구색확대상품', '스틱과자와 부드러운 딸기맛 조합으로 만들어진 메이지사의 대표 상품'],
 ['수입과자 카테고리 킬러 구색확대상품', '스틱과자와 부드러운 초콜릿맛 조합으로 만들어진 메이지사의 대표 상품'],
 ['에너자이저 스페셜티 건전지로 건강보조기구, 장난감, 자동차, 시계 등에 사용', nan],
 ['에너자이저 스페셜티 건전지로 건강보조기구, 장난감, 자동차, 시계 등에 많이 쓰임', nan],
 ['1.힘쎄고 오래가는 국내 1위 판매의 건전지 2. AAA 규격으로 상품으로 생활 필수품  3. 장난감, 도어락, 리모컨등 다용도 사용',
  '[사이드진열 Best 추천 상품] 1. AAA 규격으로 장난감 및 각종 소형 가전에 사용되어 다용도로 많은 수요가 있는 생활 필수 건전지 사이즈 2. 전 상권 필수 운영 상품 3. 에너자이저)측면진열집기세트 운영을 통한 건전지 진열 및 구색 운영'],
 ['2.1A CAR CHARGER USB 1개, 마이크로 5핀 라이트닝케이블 1개 구성.', nan],
 ['에너자이저 국내 판매 1위 건전지', '[사이드진열 Best 추천 상품] 에너자이저

# 분류 체계 매핑

In [3]:
# 거래데이터
# emart24 = ROOT / 'data/emart24Data'
# f = emart24 / '001_TRAN/IFAI_001_TRAN_00115.txt'
# f = emart24 / '007_DAYQTY/IFAI_007_DAYQTY_00115.txt'
# f = emart24 / '008_STRPUR/IFAI_008_STRPUR_00115.txt'
# f = emart24 / '009_STRORD/IFAI_009_STRORD_00115.txt'
# f = emart24 / 'IFAI_100_RCMQTY_03193.txt'
files = (ROOT / 'data/emart24Data/001_TRAN').glob('*.txt')
transaction = pd.DataFrame()
for i, file in enumerate(files):
    transaction = pd.concat([transaction, pd.read_csv(file, sep='|', low_memory=False)], 0)

# 파일 수, 데이터 건수
i + 1, transaction.shape

In [34]:
from glob import glob

In [36]:
files = (ROOT / 'data/emart24Data/001_TRAN').glob('*.txt')

In [38]:
for f in files:
    f

[]